Downloads

In [1]:
!pip install ultralytics
!pip install opencv-python
!pip install supervision

Imports

In [ ]:
### Imports
import os
from ultralytics import YOLO
import supervision as sv
import cv2
import json
import numpy as np
from tqdm import tqdm
from sports.common.team import TeamClassifier

from transformers import CLIPProcessor, CLIPModel
from PIL import Image
import torch
from transformers import AutoProcessor, SiglipVisionModel

import supervision as sv
import cv2
import json
import numpy as np
from tqdm import tqdm
import supervision as sv


# Setup

SOURCE_VIDEO_PATH = "121364_0.mp4"
OUTPUT_VIDEO_PATH = "121364_0_annotated.mp4"
JSON_OUTPUT_PATH = "121364_0_log.json"
BALL_ID = 0
SIGLIP_MODEL_PATH = 'google/siglip-base-patch16-224'

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
EMBEDDINGS_MODEL = SiglipVisionModel.from_pretrained(SIGLIP_MODEL_PATH).to(DEVICE)
EMBEDDINGS_PROCESSOR = AutoProcessor.from_pretrained(SIGLIP_MODEL_PATH)




In [5]:
player_model = YOLO('best.pt')

player_model.val(data="C:/Users/Yahya/Desktop/football_analysis-main/Nujoom/football-players-detection.v12i.yolov8/data.yaml", split='test')


Ultralytics 8.3.113  Python-3.12.4 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 3060 Laptop GPU, 6144MiB)
Model summary (fused): 112 layers, 68,127,420 parameters, 0 gradients, 257.4 GFLOPs
val: Fast image access  (ping: 0.10.0 ms, read: 334.122.5 MB/s, size: 182.0 KB)


val: Scanning C:\Users\Yahya\Desktop\football_analysis-main\Nujoom\football-players-detection.v12i.yolov8\test\labels.cache... 25 images, 0 backgrounds, 0 corrupt: 100%|██████████| 25/25 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [01:00<00:00, 30.08s/it]


                   all         25        598      0.934      0.828      0.875      0.689
                  ball         23         24          1      0.486       0.65      0.395
            goalkeeper         19         19      0.847      0.875      0.898      0.763
                player         25        499      0.977      0.988      0.994      0.864
               referee         25         56      0.911      0.964      0.959      0.732
Speed: 23.7ms preprocess, 1979.6ms inference, 0.0ms loss, 5.2ms postprocess per image
Results saved to runs\detect\val17


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x000002A4C24EAF00>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
       

In [9]:
ball_model = YOLO('ball_best.pt')

ball_model.val(data="C:/Users/Yahya/Desktop/football_analysis-main/Nujoom/football-ball-detection.v4i.yolov8/data.yaml", split='test')



Ultralytics 8.3.113  Python-3.12.4 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 3060 Laptop GPU, 6144MiB)
Model summary (fused): 72 layers, 3,006,428 parameters, 0 gradients, 8.1 GFLOPs
val: Fast image access  (ping: 0.10.1 ms, read: 5.92.2 MB/s, size: 39.9 KB)


val: Scanning C:\Users\Yahya\Desktop\football_analysis-main\Nujoom\football-ball-detection.v4i.yolov8\test\labels... 500 images, 372 backgrounds, 0 corrupt: 100%|██████████| 500/500 [00:00<00:00, 633.72it/s]

val: New cache created: C:\Users\Yahya\Desktop\football_analysis-main\Nujoom\football-ball-detection.v4i.yolov8\test\labels.cache



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 32/32 [00:13<00:00,  2.29it/s]


                   all        500        128       0.88      0.836      0.915      0.563
                  ball        128        128       0.88      0.836      0.915      0.563
Speed: 4.5ms preprocess, 12.4ms inference, 0.0ms loss, 2.1ms postprocess per image
Results saved to runs\detect\val6


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x0000023849C57770>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.0480

In [ ]:


model = player_model



ellipse_annotator = sv.EllipseAnnotator(
    color=sv.ColorPalette.from_hex(['#00BFFF', '#FF1493', '#FFD700']),
    thickness=2
)
label_annotator = sv.LabelAnnotator(
    color=sv.ColorPalette.from_hex(['#00BFFF', '#FF1493', '#FFD700']),
    text_color=sv.Color.from_hex('#000000'),
    text_position=sv.Position.BOTTOM_CENTER
)
triangle_annotator = sv.TriangleAnnotator(
    color=sv.Color.from_hex('#FFD700'),
    base=25,
    height=21,
    outline_thickness=1
)

tracker = sv.ByteTrack()
tracker.reset()

frame_generator = sv.get_video_frames_generator(SOURCE_VIDEO_PATH)
frame = next(frame_generator)

result = model.predict(frame, conf=0.1)[0]
detections = sv.Detections.from_ultralytics(result)

ball_detections = detections[detections.class_id == BALL_ID]
ball_detections.xyxy = sv.pad_boxes(xyxy=ball_detections.xyxy, px=10)

all_detections = detections[detections.class_id != BALL_ID]
all_detections = all_detections.with_nms(threshold=0.5, class_agnostic=True)
all_detections.class_id -= 1
all_detections = tracker.update_with_detections(detections=all_detections)

labels = [
    f"#{tracker_id}"
    for tracker_id
    in all_detections.tracker_id
]

annotated_frame = frame.copy()
annotated_frame = ellipse_annotator.annotate(
    scene=annotated_frame,
    detections=all_detections)
annotated_frame = label_annotator.annotate(
    scene=annotated_frame,
    detections=all_detections,
    labels=labels)
annotated_frame = triangle_annotator.annotate(
    scene=annotated_frame,
    detections=ball_detections)

sv.plot_image(annotated_frame)


0: 736x1280 4 goalkeepers, 21 players, 2 referees, 305.2ms
Speed: 12.7ms preprocess, 305.2ms inference, 1.8ms postprocess per image at shape (1, 3, 736, 1280)


<Figure size 1200x1200 with 1 Axes>

In [ ]:
from tqdm import tqdm

PLAYER_ID = 2
STRIDE = 30

frame_generator = sv.get_video_frames_generator(
    source_path=SOURCE_VIDEO_PATH, stride=STRIDE)

crops = []
for frame in tqdm(frame_generator, desc='collecting crops'):
    result = model.predict(frame, conf=0.1)[0]
    detections = sv.Detections.from_ultralytics(result)
    detections = detections.with_nms(threshold=0.5, class_agnostic=True)
    detections = detections[detections.class_id == PLAYER_ID]
    players_crops = [sv.crop_image(frame, xyxy) for xyxy in detections.xyxy]
    crops += players_crops

collecting crops: 0it [00:00, ?it/s]


0: 736x1280 4 goalkeepers, 21 players, 2 referees, 320.9ms
Speed: 11.9ms preprocess, 320.9ms inference, 1.5ms postprocess per image at shape (1, 3, 736, 1280)


collecting crops: 1it [00:00,  2.42it/s]


0: 736x1280 3 goalkeepers, 19 players, 2 referees, 100.3ms
Speed: 7.2ms preprocess, 100.3ms inference, 1.7ms postprocess per image at shape (1, 3, 736, 1280)


collecting crops: 2it [00:00,  3.82it/s]

...

collecting crops: 23it [00:03,  6.78it/s]


0: 736x1280 1 goalkeeper, 22 players, 1 referee, 99.1ms
Speed: 8.6ms preprocess, 99.1ms inference, 1.4ms postprocess per image at shape (1, 3, 736, 1280)


collecting crops: 24it [00:03,  6.79it/s]


0: 736x1280 2 goalkeepers, 22 players, 2 referees, 99.3ms
Speed: 9.1ms preprocess, 99.3ms inference, 2.5ms postprocess per image at shape (1, 3, 736, 1280)


collecting crops: 25it [00:03,  6.26it/s]


In [17]:
import torch
from transformers import AutoProcessor, SiglipVisionModel

SIGLIP_MODEL_PATH = 'google/siglip-base-patch16-224'

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
EMBEDDINGS_MODEL = SiglipVisionModel.from_pretrained(SIGLIP_MODEL_PATH).to(DEVICE)
EMBEDDINGS_PROCESSOR = AutoProcessor.from_pretrained(SIGLIP_MODEL_PATH)

Possession Calculation

In [18]:
import numpy as np
from more_itertools import chunked

BATCH_SIZE = 32

crops = [sv.cv2_to_pillow(crop) for crop in crops]
batches = chunked(crops, BATCH_SIZE)
data = []
with torch.no_grad():
    for batch in tqdm(batches, desc='embedding extraction'):
        inputs = EMBEDDINGS_PROCESSOR(images=batch, return_tensors="pt").to(DEVICE)
        outputs = EMBEDDINGS_MODEL(**inputs)
        embeddings = torch.mean(outputs.last_hidden_state, dim=1).cpu().numpy()
        data.append(embeddings)

data = np.concatenate(data)

embedding extraction: 16it [00:05,  3.10it/s]


In [19]:
import umap
from sklearn.cluster import KMeans

REDUCER = umap.UMAP(n_components=3)
CLUSTERING_MODEL = KMeans(n_clusters=2)

In [20]:
projections = REDUCER.fit_transform(data)
clusters = CLUSTERING_MODEL.fit_predict(projections)

In [21]:
import numpy as np
import supervision as sv

def resolve_goalkeepers_team_id(
    players: sv.Detections,
    goalkeepers: sv.Detections
) -> np.ndarray:
    goalkeepers_xy = goalkeepers.get_anchors_coordinates(sv.Position.BOTTOM_CENTER)
    players_xy = players.get_anchors_coordinates(sv.Position.BOTTOM_CENTER)
    team_0_centroid = players_xy[players.class_id == 0].mean(axis=0)
    team_1_centroid = players_xy[players.class_id == 1].mean(axis=0)
    goalkeepers_team_id = []
    for goalkeeper_xy in goalkeepers_xy:
        dist_0 = np.linalg.norm(goalkeeper_xy - team_0_centroid)
        dist_1 = np.linalg.norm(goalkeeper_xy - team_1_centroid)
        goalkeepers_team_id.append(0 if dist_0 < dist_1 else 1)

    return np.array(goalkeepers_team_id)

In [ ]:
import supervision as sv
from tqdm import tqdm
from sports.common.team import TeamClassifier


frame_generator = sv.get_video_frames_generator(
    source_path=SOURCE_VIDEO_PATH, stride=STRIDE)

crops = []
for frame in tqdm(frame_generator, desc='collecting crops'):
    result = model.predict(frame, conf=0.1)[0]
    detections = sv.Detections.from_ultralytics(result)
    players_detections = detections[detections.class_id == PLAYER_ID]
    players_crops = [sv.crop_image(frame, xyxy) for xyxy in detections.xyxy]
    crops += players_crops

team_classifier = TeamClassifier(device="cuda")
team_classifier.fit(crops)

collecting crops: 0it [00:00, ?it/s]


0: 736x1280 4 goalkeepers, 21 players, 2 referees, 306.8ms
Speed: 12.0ms preprocess, 306.8ms inference, 1.9ms postprocess per image at shape (1, 3, 736, 1280)


collecting crops: 1it [00:00,  2.55it/s]


0: 736x1280 3 goalkeepers, 19 players, 2 referees, 101.8ms
Speed: 7.4ms preprocess, 101.8ms inference, 1.8ms postprocess per image at shape (1, 3, 736, 1280)


collecting crops: 2it [00:00,  3.96it/s]

...

collecting crops: 23it [00:03,  6.81it/s]


0: 736x1280 1 goalkeeper, 22 players, 1 referee, 99.2ms
Speed: 9.1ms preprocess, 99.2ms inference, 1.4ms postprocess per image at shape (1, 3, 736, 1280)


collecting crops: 24it [00:03,  6.84it/s]


0: 736x1280 2 goalkeepers, 22 players, 2 referees, 98.5ms
Speed: 8.0ms preprocess, 98.5ms inference, 1.5ms postprocess per image at shape (1, 3, 736, 1280)


collecting crops: 25it [00:03,  6.32it/s]
Embedding extraction: 19it [00:06,  3.02it/s]


In [ ]:
import supervision as sv

BALL_ID = 0
GOALKEEPER_ID = 1
PLAYER_ID = 2
REFEREE_ID = 3

ellipse_annotator = sv.EllipseAnnotator(
    color=sv.ColorPalette.from_hex(['#00BFFF', '#FF1493', '#FFD700']),
    thickness=2
)
label_annotator = sv.LabelAnnotator(
    color=sv.ColorPalette.from_hex(['#00BFFF', '#FF1493', '#FFD700']),
    text_color=sv.Color.from_hex('#000000'),
    text_position=sv.Position.BOTTOM_CENTER
)
triangle_annotator = sv.TriangleAnnotator(
    color=sv.Color.from_hex('#FFD700'),
    base=25,
    height=21,
    outline_thickness=1
)

tracker = sv.ByteTrack()
tracker.reset()

frame_generator = sv.get_video_frames_generator(SOURCE_VIDEO_PATH)
frame = next(frame_generator)

result = model.predict(frame, conf=0.1)[0]
detections = sv.Detections.from_ultralytics(result)

ball_detections = detections[detections.class_id == BALL_ID]
ball_detections.xyxy = sv.pad_boxes(xyxy=ball_detections.xyxy, px=10)

all_detections = detections[detections.class_id != BALL_ID]
all_detections = all_detections.with_nms(threshold=0.5, class_agnostic=True)
all_detections = tracker.update_with_detections(detections=all_detections)

goalkeepers_detections = all_detections[all_detections.class_id == GOALKEEPER_ID]
players_detections = all_detections[all_detections.class_id == PLAYER_ID]
referees_detections = all_detections[all_detections.class_id == REFEREE_ID]

players_crops = [sv.crop_image(frame, xyxy) for xyxy in players_detections.xyxy]
players_detections.class_id = team_classifier.predict(players_crops)

goalkeepers_detections.class_id = resolve_goalkeepers_team_id(
    players_detections, goalkeepers_detections)

referees_detections.class_id -= 1

all_detections = sv.Detections.merge([
    players_detections, goalkeepers_detections, referees_detections])

labels = [
    f"#{tracker_id}"
    for tracker_id
    in all_detections.tracker_id
]

all_detections.class_id = all_detections.class_id.astype(int)

annotated_frame = frame.copy()
annotated_frame = ellipse_annotator.annotate(
    scene=annotated_frame,
    detections=all_detections)
annotated_frame = label_annotator.annotate(
    scene=annotated_frame,
    detections=all_detections,
    labels=labels)
annotated_frame = triangle_annotator.annotate(
    scene=annotated_frame,
    detections=ball_detections)

sv.plot_image(annotated_frame)


0: 736x1280 4 goalkeepers, 21 players, 2 referees, 377.9ms
Speed: 15.0ms preprocess, 377.9ms inference, 2.1ms postprocess per image at shape (1, 3, 736, 1280)


Embedding extraction: 1it [00:00,  5.05it/s]


<Figure size 1200x1200 with 1 Axes>

In [29]:
import supervision as sv
import cv2
import numpy as np
import json
from tqdm import tqdm

# === Paths ===
SOURCE_VIDEO_PATH = "121364_0.mp4"
OUTPUT_VIDEO_PATH = "121364_0_annotated.mp4"
JSON_OUTPUT_PATH = "121364_0_log.json"

# === IDs ===
BALL_ID = 0
GOALKEEPER_ID = 1
PLAYER_ID = 2
REFEREE_ID = 3

# === Annotators ===
ellipse_annotator = sv.EllipseAnnotator(
    color=sv.ColorPalette.from_hex(['#00BFFF', '#FF1493', '#FFD700']),
    thickness=2
)
label_annotator = sv.LabelAnnotator(
    color=sv.ColorPalette.from_hex(['#00BFFF', '#FF1493', '#FFD700']),
    text_color=sv.Color.from_hex('#000000'),
    text_position=sv.Position.BOTTOM_CENTER
)
triangle_annotator = sv.TriangleAnnotator(
    color=sv.Color.from_hex('#FFD700'),
    base=25,
    height=21,
    outline_thickness=1
)

# === Video Setup ===
cap = cv2.VideoCapture(SOURCE_VIDEO_PATH)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
writer = cv2.VideoWriter(OUTPUT_VIDEO_PATH, cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))
frame_generator = sv.get_video_frames_generator(SOURCE_VIDEO_PATH)

tracker = sv.ByteTrack()
tracker.reset()
log_data = []

# === Frame Loop ===
for frame_index, frame in tqdm(enumerate(frame_generator), total=total_frames):
    result = model.predict(frame, conf=0.1)[0]
    detections = sv.Detections.from_ultralytics(result)

    # Ball
    ball_detections = detections[detections.class_id == BALL_ID]
    ball_detections.xyxy = sv.pad_boxes(xyxy=ball_detections.xyxy, px=10)

    # All other detections
    all_detections = detections[detections.class_id != BALL_ID]
    all_detections = all_detections.with_nms(threshold=0.5, class_agnostic=True)
    all_detections = tracker.update_with_detections(detections=all_detections)

    # Split classes
    goalkeepers_detections = all_detections[all_detections.class_id == GOALKEEPER_ID]
    players_detections = all_detections[all_detections.class_id == PLAYER_ID]
    referees_detections = all_detections[all_detections.class_id == REFEREE_ID]

    # Team classification
    players_crops = [sv.crop_image(frame, xyxy) for xyxy in players_detections.xyxy]
    players_detections.class_id = team_classifier.predict(players_crops)
    goalkeepers_detections.class_id = resolve_goalkeepers_team_id(players_detections, goalkeepers_detections)
    referees_detections.class_id -= 1

    # Merge and annotate
    all_detections = sv.Detections.merge([
        players_detections, goalkeepers_detections, referees_detections
    ])
    all_detections.class_id = all_detections.class_id.astype(int)

    labels = [f"#{tracker_id}" for tracker_id in all_detections.tracker_id]

    annotated_frame = frame.copy()
    annotated_frame = ellipse_annotator.annotate(scene=annotated_frame, detections=all_detections)
    annotated_frame = label_annotator.annotate(scene=annotated_frame, detections=all_detections, labels=labels)
    annotated_frame = triangle_annotator.annotate(scene=annotated_frame, detections=ball_detections)
    writer.write(annotated_frame)

    # === Frame Log ===
    frame_log = {
        "frame": frame_index,
        "timestamp_sec": round(frame_index / fps, 2),
        "detections": []
    }

    for tracker_id, class_id, xyxy in zip(all_detections.tracker_id, all_detections.class_id, all_detections.xyxy):
        label = ["player", "GK", "referee"][class_id] if class_id < 3 else "unknown"
        team_id = int(class_id == 0)  # crude: players and GK only
        frame_log["detections"].append({
            "label": label,
            "id": int(tracker_id),
            "team_id": team_id,
            "box": list(map(float, xyxy))
        })

    for xyxy in ball_detections.xyxy:
        frame_log["detections"].append({
            "label": "ball",
            "id": None,
            "team_id": None,
            "box": list(map(float, xyxy))
        })

    log_data.append(frame_log)

# === Finalize ===
cap.release()
writer.release()

with open(JSON_OUTPUT_PATH, "w") as f:
    json.dump(log_data, f, indent=2)

print(f"✅ Annotated video saved at: {OUTPUT_VIDEO_PATH}")
print(f"✅ Detection log saved at: {JSON_OUTPUT_PATH}")


  0%|          | 0/750 [00:00<?, ?it/s]


0: 736x1280 4 goalkeepers, 21 players, 2 referees, 385.0ms
Speed: 32.5ms preprocess, 385.0ms inference, 11.2ms postprocess per image at shape (1, 3, 736, 1280)


Embedding extraction: 1it [00:00,  3.39it/s]
  0%|          | 1/750 [00:00<12:20,  1.01it/s]


0: 736x1280 4 goalkeepers, 21 players, 2 referees, 104.7ms
Speed: 7.9ms preprocess, 104.7ms inference, 1.9ms postprocess per image at shape (1, 3, 736, 1280)


Embedding extraction: 1it [00:00,  3.85it/s]
  0%|          | 2/750 [00:01<08:27,  1.48it/s]

...


0: 736x1280 1 goalkeeper, 22 players, 2 referees, 107.5ms
Speed: 6.5ms preprocess, 107.5ms inference, 1.7ms postprocess per image at shape (1, 3, 736, 1280)


Embedding extraction: 1it [00:00,  4.17it/s]
100%|█████████▉| 749/750 [04:49<00:00,  2.30it/s]


0: 736x1280 1 goalkeeper, 21 players, 2 referees, 116.7ms
Speed: 7.4ms preprocess, 116.7ms inference, 2.1ms postprocess per image at shape (1, 3, 736, 1280)


Embedding extraction: 1it [00:00,  4.24it/s]
100%|██████████| 750/750 [04:50<00:00,  2.58it/s]


✅ Annotated video saved at: 121364_0_annotated.mp4
✅ Detection log saved at: 121364_0_log.json


In [30]:
import json
import numpy as np

# Load detection log
with open("121364_0_log.json", "r") as f:
    log_data = json.load(f)

# Define possession threshold (in pixels, tweak as needed)
possession_threshold = 50

# Loop through each frame and compute possession
for frame in log_data:
    ball_box = None
    players = []

    for det in frame["detections"]:
        if det["label"] == "ball":
            ball_box = det["box"]
        elif det["label"] == "player":
            players.append({"id": det["id"], "box": det["box"]})

    possessor_id = None
    min_distance = float("inf")

    if ball_box:
        ball_center = np.array([
            (ball_box[0] + ball_box[2]) / 2,
            (ball_box[1] + ball_box[3]) / 2
        ])

        for player in players:
            player_box = player["box"]
            player_center = np.array([
                (player_box[0] + player_box[2]) / 2,
                (player_box[1] + player_box[3]) / 2
            ])
            distance = np.linalg.norm(player_center - ball_center)

            if distance < possession_threshold and distance < min_distance:
                min_distance = distance
                possessor_id = player["id"]

    frame["possession"] = possessor_id  # Will be None if no one has the ball

# Save updated log
output_path = "121364_0_log_with_possession.json"
with open(output_path, "w") as f:
    json.dump(log_data, f, indent=2)

print(f"✅ Possession data added and saved to: {output_path}")


✅ Possession data added and saved to: 121364_0_log_with_possession.json
